In [1]:
!pwd

/mnt/d/software/cp2k_mc_workflow/testing/nitrogen_mc


In [1]:
import mbuild as mb
import foyer
import parmed as pmd
import mbuild.formats.charmm_writer as mf_charmm
import mosdef_cp2k_writer
from constrainmol import ConstrainedMolecule
from foyer import Forcefield
# doing simulation with hoomd
import mdtraj as md
import sys
print(sys.executable)
import os
print(os.getcwd())
print(sys.path)
import gsd.hoomd
import matplotlib.pyplot as plt
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import hoomd
import hoomd.md
import hoomd.group
import mbuild as mb
import foyer


/home/rs/anaconda3/envs/cp2k_mc/bin/python
/mnt/d/software/MC-workflow/nitrogen-mc/increase_temp/130K
['/mnt/d/software/MC-workflow/nitrogen-mc/increase_temp/130K', '/home/rs/anaconda3/envs/cp2k_mc/lib/python38.zip', '/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8', '/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/lib-dynload', '', '/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages', '/mnt/d/software/MC-workflow/mbuild', '/mnt/d/software/MC-workflow/mosdef_cp2k_writer', '/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/IPython/extensions', '/home/rs/.ipython']


In [2]:
# creating biasing potential file
!rm -r trajectories
!mkdir trajectories

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
T=130
ll=1.820963
lv=1.827094
box1n=57
box2n=23
nitrogen_res_name = 'N2'
FF_file_nitrogen = 'dinitrogen_ff.xml'
nitrogen_FF = Forcefield(forcefield_files=FF_file_nitrogen)
nitrogen_mb = mb.load('N#N', smiles=True)

nitrogen_mb.name = nitrogen_res_name

nitrogen_typed=nitrogen_FF.apply(nitrogen_mb)

print(nitrogen_mb.xyz)

constrain_mol = ConstrainedMolecule(nitrogen_typed)
constrain_mol.solve()
nitrogen_optimized_mb=mb.clone(nitrogen_mb)
nitrogen_optimized_mb.xyz=constrain_mol.xyz/10
print(nitrogen_optimized_mb.xyz)

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:448: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:460: UserWarning: No force field name found in force field XML file.
  warnings.warn(


[[ 0.09427732 -0.00699514  0.00565158]
 [ 0.23627732 -0.00699514  0.00565158]]


/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule N#N	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


[[ 0.10922732 -0.00699514  0.00565158]
 [ 0.22132732 -0.00699514  0.00565158]]


In [4]:
nitrogen_box_bias  = mb.fill_box(compound=[nitrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
FF_Dict = {nitrogen_mb.name:FF_file_nitrogen }

residues_List = [nitrogen_mb.name ]




nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(nitrogen_box_bias ,
                            'nitrogen_bias',
                            FF_filename ="nitrogen_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'N2': 'dinitrogen_ff.xml'}, residues = ['N2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'N2': 'dinitrogen_ff.xml'}
coulomb14scale from compound = {'N2': 0.0}
lj14scale from compound = {'N2': 0.0}
unique_types = ['N_N2']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {0: 'N_N2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'N2': 'dinitrogen_ff.xml'}, residues = ['N2']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['N2']
fix_residue = None
fix_res

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


In [6]:
print('Running: filling liquid box')

box = mb.box.Box([ll, ll, ll])

box_sparse=mb.box.Box([ll*10 ,ll*10, ll*10])
nitrogen_box_liq  = mb.fill_box(compound=[nitrogen_optimized_mb  ],
                            n_compounds=box1n ,
                            box=box_sparse)
print('Completed: filling liquid box')
nitrogen_box_liq_pmd=nitrogen_box_liq.to_parmed(box=box_sparse)

dinitrogen_ff = foyer.Forcefield(forcefield_files='dinitrogen_ff.xml')

struc = dinitrogen_ff.apply(nitrogen_box_liq_pmd,assert_bond_params=True,
                 assert_angle_params=True,
                 assert_dihedral_params=False)
for i in range(len(struc.atoms)):

    struc.atoms[i].type=struc.atoms[i].element_name
    struc.atoms[i].atom_type.name=struc.atoms[i].element_name

    
create_hoomd_simulation(struc,r_cut=1.2,auto_scale=False,ref_distance=1)
_all = hoomd.group.all()
hoomd.md.integrate.mode_standard(dt=0.0001)
integrator = hoomd.md.integrate.nvt(group=_all, kT= 0.00831446*T, tau=1)
hoomd.dump.gsd("trajectories/start-shrink.gsd", period=None, group=_all, overwrite=True)
hoomd.dump.gsd("trajectories/traj-shrink.gsd", period=1e5, group=_all, phase=0, overwrite=True)

integrator.randomize_velocities(seed=42);

resize=hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,ll*100), (1e4,ll*10)], zero=0));


hoomd.run(1e4+1)
resize.disable()

hoomd.analyze.log(filename="log-output_nvt.log",
                  quantities=['kinetic_energy','potential_energy','temperature','pressure','lx','ly','lz'],
                  period=100,
                  overwrite=True);

hoomd.dump.gsd("trajectories/traj-nvtmd.gsd", period=1e3, group=_all, phase=0, overwrite=True)

hoomd.run(1e4)

gsd_restart = hoomd.dump.gsd("trajectories/out-shrink.gsd", period=None, group=_all, overwrite=True);
gsd_restart.write_restart()
integrator.disable

Running: filling liquid box


/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Completed: filling liquid box
HOOMD-blue 2.9.3 CUDA (11.0) DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 12/16/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
CUDA driver version is insufficient for CUDA runtime version
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 114 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 114
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
Processing LJ and QQ
notice(2): Group "charged" created containing 0 particles
No charged groups found, ignoring electrostatics
Processing harmonic bonds
HOOMD Simulati

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


Time 00:00:10 | Step 2501 / 10001 | TPS 246.586 | ETA 00:00:30
Time 00:00:14 | Step 10001 / 10001 | TPS 1854.96 | ETA 00:00:00
Average TPS: 704.949
---------
-- Neighborlist stats:
0 normal updates / 101 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 2 / n_neigh_avg: 0.184211
shortest rebuild period: 100
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 0 / n_max: 2 / n_avg: 0.0856499
** run complete **
** starting run **
Time 00:00:14 | Step 20001 / 20001 | TPS 73077 | ETA 00:00:00
Average TPS: 72523.2
---------
-- Neighborlist stats:
0 normal updates / 100 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 1 / n_neigh_avg: 0.0526316
shortest rebuild period: 100
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 0 / n_max: 2 / n_avg: 0.0856499
** run complete **


<bound method _integration_method.disable of <hoomd.md.integrate.nvt object at 0x7fde8824a730>>

In [7]:
import MDAnalysis as mda
u = mda.Universe("trajectories/traj-nvtmd.gsd")
print(len(u.trajectory))
u.trajectory[-1]
N = u.select_atoms("name N", updating=True)
N.write("nitrogen_box_liq.xyz")


10


/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
#run the vap box file

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
#creating bias_template.inp file

from mosdef_cp2k_writer.classes import SIM as sim

mySim = sim.SIM()
mySim.GLOBAL.RUN_TYPE = "MC"
mySim.GLOBAL.PROJECT_NAME="bias_template"
mySim.GLOBAL.PRINT_LEVEL="LOW"

mySim.MOTION.MC.ENSEMBLE="TRADITIONAL"
mySim.MOTION.MC.TEMPERATURE =T
mySim.MOTION.MC.PRESSURE= 13.8
mySim.MOTION.MC.IPRINT= 1
mySim.MOTION.MC.LBIAS='TRUE'
mySim.MOTION.MC.LSTOP='FALSE'
mySim.MOTION.MC.NMOVES=8
mySim.MOTION.MC.NSWAPMOVES =640
mySim.MOTION.MC.NSTEP= 1
mySim.MOTION.MC.RESTART='FALSE'

mySim.MOTION.MC.ETA= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.PMSWAP= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.PMTRAION= 0.20
mySim.MOTION.MC.MOVE_PROBABILITIES.PMTRANS= 0.60
mySim.MOTION.MC.MOVE_PROBABILITIES.PMVOLUME= 0.02
mySim.MOTION.MC.MOVE_PROBABILITIES.PMHMC= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.PMAVBMC= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMAVBMC_MOL=[1]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMSWAP_MOL =[1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMTRAION_MOL= [1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMTRANS_MOL= [1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMROT_MOL =[1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.BOX_PROBABILITIES.PMVOL_BOX =1.0
mySim.MOTION.MC.MOVE_PROBABILITIES.BOX_PROBABILITIES.PMHMC_BOX= 1.0
mySim.MOTION.MC.MOVE_UPDATES.IUPTRANS =100
mySim.MOTION.MC.MOVE_UPDATES.IUPVOLUME= 100
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMDIHEDRAL= [3.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMANGLE= [3.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMBOND =[0.074]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMROT =[26.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMTRANS=[ 0.38]
mySim.MOTION.MC.MAX_DISPLACEMENTS.BOX_DISPLACEMENTS.RMVOLUME =100.5
mySim.MOTION.MC.AVBMC.AVBMC_ATOM =[1]
mySim.MOTION.MC.AVBMC.AVBMC_RMIN =[1.0]
mySim.MOTION.MC.AVBMC.AVBMC_RMAX =[5.0]
mySim.MOTION.MC.AVBMC.PBIAS= [0.5]

mySim.FORCE_EVAL.METHOD="FIST"
mySim.FORCE_EVAL.MM.FORCEFIELD.PARMTYPE="CHM"
mySim.FORCE_EVAL.MM.FORCEFIELD.PARM_FILE_NAME="nitrogen_charmm_bias.inp"
mySim.FORCE_EVAL.MM.FORCEFIELD.SPLINE.EMAX_SPLINE=10000 
mySim.FORCE_EVAL.MM.POISSON.EWALD.EWALD_TYPE="EWALD"
mySim.FORCE_EVAL.MM.POISSON.EWALD.EWALD_ACCURACY=1e-6
mySim.FORCE_EVAL.MM.POISSON.EWALD.GMAX=25

mySim.FORCE_EVAL.SUBSYS.CELL.ABC="10 10 10"
mySim.FORCE_EVAL.SUBSYS.CELL.PERIODIC="XYZ"
mySim.FORCE_EVAL.SUBSYS.CELL.SYMMETRY="CUBIC"
mySim.FORCE_EVAL.SUBSYS.COORD.DEFAULT_KEYWORD="nitrogen_bias_coord.xyz"
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.CONN_FILE_FORMAT="MOL_SET"
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.init_molecules(1)
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.MOLECULE[1].NMOL=box1n
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.MOLECULE[1].CONN_FILE_NAME="nitrogen_bias.psf"
mySim.write_changeLog(fn="changeLog.out")
mySim.write_errorLog()
mySim.write_inputFile(fn='bias_template.inp')


/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
!cat bias_template.inp


&FORCE_EVAL
  METHOD        FIST
  &MM        
    &FORCEFIELD        
      PARM_FILE_NAME        nitrogen_charmm_bias.inp
      PARMTYPE        CHM
      &SPLINE        
        EMAX_SPLINE        10000
      &END SPLINE        
    &END FORCEFIELD        
    &POISSON       
      &EWALD        
        EWALD_ACCURACY        1e-06
        EWALD_TYPE        EWALD
        GMAX        25
      &END EWALD        
    &END POISSON        
  &END MM        
  &DFT        
    &MGRID        
    &END MGRID        
    &QS        
    &END QS        
    &POISSON        
      &EWALD        
      &END EWALD        
    &END POISSON        
    &PRINT         
    &END PRINT        
    &SCF        
    &END SCF        
    &XC        
      &VDW_POTENTIAL        
        &PAIR_POTENTIAL        
        &END PAIR_POTENTIAL        
      &END VDW_POTENTIAL        
      &XC_GRID        
      &END XC_GRID        
    &END XC        
  &END DFT        
  &SUBSYS        
    &CELL        
    

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
#creating first box input file

from mosdef_cp2k_writer.classes import SIM as sim

mySim = sim.SIM()
mySim.GLOBAL.RUN_TYPE = "MC"
mySim.GLOBAL.PROJECT_NAME="N2_MC"
mySim.GLOBAL.PRINT_LEVEL="LOW"

mySim.MOTION.MC.ENSEMBLE="GEMC_NVT"
mySim.MOTION.MC.TEMPERATURE =T
mySim.MOTION.MC.PRESSURE= 1.013
mySim.MOTION.MC.IPRINT= 1
mySim.MOTION.MC.LBIAS='TRUE'
mySim.MOTION.MC.LSTOP='FALSE'
mySim.MOTION.MC.NMOVES=8
mySim.MOTION.MC.NSWAPMOVES =640
mySim.MOTION.MC.NSTEP= 120
mySim.MOTION.MC.RESTART='FALSE'
mySim.MOTION.MC.BOX2_FILE_NAME="GEMC_NVT_box2.inp"
mySim.MOTION.MC.RESTART_FILE_NAME="mc_restart_1"

mySim.MOTION.MC.ETA= 0.0
#mySim.MOTION.MC.VIRIAL_TEMPS=300

mySim.MOTION.MC.MOVE_PROBABILITIES.PMSWAP= 0.4
mySim.MOTION.MC.MOVE_PROBABILITIES.PMTRAION= 0.50
mySim.MOTION.MC.MOVE_PROBABILITIES.PMTRANS= 0.750
mySim.MOTION.MC.MOVE_PROBABILITIES.PMVOLUME= 0.05
mySim.MOTION.MC.MOVE_PROBABILITIES.PMHMC= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.PMAVBMC= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMAVBMC_MOL=[1]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMSWAP_MOL =[1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMTRAION_MOL= [1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMTRANS_MOL= [1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMROT_MOL =[1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.BOX_PROBABILITIES.PMVOL_BOX =1.0
mySim.MOTION.MC.MOVE_PROBABILITIES.BOX_PROBABILITIES.PMHMC_BOX= 1.0
mySim.MOTION.MC.MOVE_UPDATES.IUPTRANS =20
mySim.MOTION.MC.MOVE_UPDATES.IUPVOLUME= 20
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMDIHEDRAL= [3.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMANGLE= [3.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMBOND =[0.074]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMROT =[26.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMTRANS=[ 0.38]
mySim.MOTION.MC.MAX_DISPLACEMENTS.BOX_DISPLACEMENTS.RMVOLUME =100.5
mySim.MOTION.MC.AVBMC.AVBMC_ATOM =[1]
mySim.MOTION.MC.AVBMC.AVBMC_RMIN =[1.0]
mySim.MOTION.MC.AVBMC.AVBMC_RMAX =[5.0]
mySim.MOTION.MC.AVBMC.PBIAS= [0.5]

mySim.FORCE_EVAL.METHOD="Quickstep"
mySim.FORCE_EVAL.DFT.BASIS_SET_FILE_NAME="BASIS_MOLOPT"
mySim.FORCE_EVAL.DFT.POTENTIAL_FILE_NAME="GTH_POTENTIALS"
mySim.FORCE_EVAL.DFT.MGRID.CUTOFF=600
mySim.FORCE_EVAL.DFT.MGRID.REL_CUTOFF=60
mySim.FORCE_EVAL.DFT.SCF.SCF_GUESS="RESTART"
mySim.FORCE_EVAL.DFT.SCF.MAX_SCF=20
mySim.FORCE_EVAL.DFT.SCF.EPS_SCF=1e-5
mySim.FORCE_EVAL.DFT.SCF.OT.SECTION_PARAMETERS='.TRUE.'
mySim.FORCE_EVAL.DFT.SCF.OT.MINIMIZER="DIIS"
mySim.FORCE_EVAL.DFT.SCF.OT.PRECONDITIONER ="FULL_SINGLE_INVERSE"

mySim.FORCE_EVAL.DFT.XC.XC_FUNCTIONAL.SECTION_PARAMETERS="PBE"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.POTENTIAL_TYPE="PAIR_POTENTIAL"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.R_CUTOFF=16
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.TYPE="DFTD3"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.CALCULATE_C9_TERM =".TRUE."
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.REFERENCE_C9_TERM =".TRUE."
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.LONG_RANGE_CORRECTION =".TRUE."
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.REFERENCE_FUNCTIONAL="PBE"

mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.PARAMETER_FILE_NAME ="dftd3.dat"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.EPS_CN =1e-6
mySim.FORCE_EVAL.DFT.XC.XC_GRID.XC_DERIV="SPLINE2"
mySim.FORCE_EVAL.DFT.XC.XC_GRID.XC_SMOOTH_RHO="NONE"


mySim.FORCE_EVAL.SUBSYS.CELL.ABC="{} {} {} ".format(ll*10,ll*10,ll*10)
mySim.FORCE_EVAL.SUBSYS.CELL.CELL_REF.ABC="{} {} {} ".format(ll*10,ll*10,ll*10)
mySim.FORCE_EVAL.SUBSYS.CELL.PERIODIC="XYZ"
mySim.FORCE_EVAL.SUBSYS.CELL.SYMMETRY="CUBIC"
mySim.FORCE_EVAL.SUBSYS.COORD.DEFAULT_KEYWORD="nitrogen_box_liq.xyz"
mySim.FORCE_EVAL.SUBSYS.init_atoms(1)
mySim.FORCE_EVAL.SUBSYS.KIND[1].SECTION_PARAMETERS="N"
mySim.FORCE_EVAL.SUBSYS.KIND[1].BASIS_SET="DZVP-MOLOPT-SR-GTH"
mySim.FORCE_EVAL.SUBSYS.KIND[1].POTENTIAL="GTH-PBE"


mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.CONN_FILE_FORMAT="MOL_SET"
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.init_molecules(1)
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.MOLECULE[1].NMOL=box1n
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.MOLECULE[1].CONN_FILE_NAME="nitrogen_bias.psf"

mySim.write_changeLog(fn="changeLog.out")
mySim.write_errorLog()
mySim.write_inputFile(fn='GEMC_NVT_box1.inp')


/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
#creating second box input file

from mosdef_cp2k_writer.classes import SIM as sim

mySim = sim.SIM()
mySim.GLOBAL.RUN_TYPE = "MC"
mySim.GLOBAL.PROJECT_NAME="N2_MC"
mySim.GLOBAL.PRINT_LEVEL="LOW"

mySim.MOTION.MC.ENSEMBLE="GEMC_NVT"
mySim.MOTION.MC.TEMPERATURE =T
mySim.MOTION.MC.PRESSURE= 1.013
mySim.MOTION.MC.IPRINT= 1
mySim.MOTION.MC.LBIAS='TRUE'
mySim.MOTION.MC.LSTOP='FALSE'
mySim.MOTION.MC.NMOVES=8
mySim.MOTION.MC.NSWAPMOVES =640
mySim.MOTION.MC.NSTEP= 120
mySim.MOTION.MC.RESTART='FALSE'
mySim.MOTION.MC.BOX2_FILE_NAME="GEMC_NVT_box1.inp"
mySim.MOTION.MC.RESTART_FILE_NAME="mc_restart_2"

mySim.MOTION.MC.ETA= 0.0
#mySim.MOTION.MC.VIRIAL_TEMPS=300

mySim.MOTION.MC.MOVE_PROBABILITIES.PMSWAP= 0.4
mySim.MOTION.MC.MOVE_PROBABILITIES.PMTRAION= 0.50
mySim.MOTION.MC.MOVE_PROBABILITIES.PMTRANS= 0.750
mySim.MOTION.MC.MOVE_PROBABILITIES.PMVOLUME= 0.05
mySim.MOTION.MC.MOVE_PROBABILITIES.PMHMC= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.PMAVBMC= 0.0
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMAVBMC_MOL=[1]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMSWAP_MOL =[1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMTRAION_MOL= [1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMTRANS_MOL= [1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.MOL_PROBABILITIES.PMROT_MOL =[1.0]
mySim.MOTION.MC.MOVE_PROBABILITIES.BOX_PROBABILITIES.PMVOL_BOX =1.0
mySim.MOTION.MC.MOVE_PROBABILITIES.BOX_PROBABILITIES.PMHMC_BOX= 1.0
mySim.MOTION.MC.MOVE_UPDATES.IUPTRANS =20
mySim.MOTION.MC.MOVE_UPDATES.IUPVOLUME= 20
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMDIHEDRAL= [3.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMANGLE= [3.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMBOND =[0.074]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMROT =[26.0]
mySim.MOTION.MC.MAX_DISPLACEMENTS.MOL_DISPLACEMENTS.RMTRANS=[ 0.38]
mySim.MOTION.MC.MAX_DISPLACEMENTS.BOX_DISPLACEMENTS.RMVOLUME =100.5
mySim.MOTION.MC.AVBMC.AVBMC_ATOM =[1]
mySim.MOTION.MC.AVBMC.AVBMC_RMIN =[1.0]
mySim.MOTION.MC.AVBMC.AVBMC_RMAX =[5.0]
mySim.MOTION.MC.AVBMC.PBIAS= [0.5]

mySim.FORCE_EVAL.METHOD="Quickstep"
mySim.FORCE_EVAL.DFT.BASIS_SET_FILE_NAME="BASIS_MOLOPT"
mySim.FORCE_EVAL.DFT.POTENTIAL_FILE_NAME="GTH_POTENTIALS"
mySim.FORCE_EVAL.DFT.MGRID.CUTOFF=600
mySim.FORCE_EVAL.DFT.MGRID.REL_CUTOFF=60
mySim.FORCE_EVAL.DFT.SCF.SCF_GUESS="RESTART"
mySim.FORCE_EVAL.DFT.SCF.MAX_SCF=20
mySim.FORCE_EVAL.DFT.SCF.EPS_SCF=1e-5
mySim.FORCE_EVAL.DFT.SCF.OT.SECTION_PARAMETERS='.TRUE.'
mySim.FORCE_EVAL.DFT.SCF.OT.MINIMIZER="DIIS"
mySim.FORCE_EVAL.DFT.SCF.OT.PRECONDITIONER ="FULL_SINGLE_INVERSE"

mySim.FORCE_EVAL.DFT.XC.XC_FUNCTIONAL.SECTION_PARAMETERS="PBE"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.POTENTIAL_TYPE="PAIR_POTENTIAL"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.R_CUTOFF=16
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.TYPE="DFTD3"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.CALCULATE_C9_TERM =".TRUE."
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.REFERENCE_C9_TERM =".TRUE."
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.LONG_RANGE_CORRECTION =".TRUE."
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.REFERENCE_FUNCTIONAL="PBE"

mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.PARAMETER_FILE_NAME ="dftd3.dat"
mySim.FORCE_EVAL.DFT.XC.VDW_POTENTIAL.PAIR_POTENTIAL.EPS_CN =1e-6
mySim.FORCE_EVAL.DFT.XC.XC_GRID.XC_DERIV="SPLINE2"
mySim.FORCE_EVAL.DFT.XC.XC_GRID.XC_SMOOTH_RHO="NONE"


mySim.FORCE_EVAL.SUBSYS.CELL.ABC="{} {} {} ".format(lv*10,lv*10,lv*10)
mySim.FORCE_EVAL.SUBSYS.CELL.CELL_REF.ABC="{} {} {} ".format(lv*10,lv*10,lv*10)
mySim.FORCE_EVAL.SUBSYS.CELL.PERIODIC="XYZ"
mySim.FORCE_EVAL.SUBSYS.CELL.SYMMETRY="CUBIC"
mySim.FORCE_EVAL.SUBSYS.COORD.DEFAULT_KEYWORD="nitrogen_box_vap.xyz"
mySim.FORCE_EVAL.SUBSYS.init_atoms(1)
mySim.FORCE_EVAL.SUBSYS.KIND[1].SECTION_PARAMETERS="N"
mySim.FORCE_EVAL.SUBSYS.KIND[1].BASIS_SET="DZVP-MOLOPT-SR-GTH"
mySim.FORCE_EVAL.SUBSYS.KIND[1].POTENTIAL="GTH-PBE"


mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.CONN_FILE_FORMAT="MOL_SET"
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.init_molecules(1)
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.MOLECULE[1].NMOL=box2n
mySim.FORCE_EVAL.SUBSYS.TOPOLOGY.MOL_SET.MOLECULE[1].CONN_FILE_NAME="nitrogen_bias.psf"
mySim.write_changeLog(fn="changeLog.out")
mySim.write_errorLog()
mySim.write_inputFile(fn='GEMC_NVT_box2.inp')


/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#remove first two lines
with open('nitrogen_bias_coord.xyz', 'r') as fin:
    data = fin.read().splitlines(True)
with open('nitrogen_bias_coord.xyz', 'w') as fout:
    fout.writelines(data[2:])

    
    #remove first two lines
with open('nitrogen_box_liq.xyz', 'r') as fin:
    data = fin.read().splitlines(True)
with open('nitrogen_box_liq.xyz', 'w') as fout:
    fout.writelines(data[2:])
    
    
#remove first two lines
with open('nitrogen_box_vap.xyz', 'r') as fin:
    data = fin.read().splitlines(True)
with open('nitrogen_box_vap.xyz', 'w') as fout:
    fout.writelines(data[2:])
    

    


    



    

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!cp2k.popt -i  GEMC_NVT_box1.inp -i GEMC_NVT_box2.inp

In [ ]:
nitrogen_box_liq_85K

In [ ]:

nitrogen_box_vap = mb.fill_box(compound=[nitrogen_optimized_mb],
                            n_compounds=box2n ,
                            box=[lv,lv,lv])
print('Completed: filling vapor box')
nitrogen_box_vap.save("nitrogen_box_vap.xyz",overwrite=True)
print('Running: GOMC FF file, and the psf and pdb files for 85 K box')
mf_charmm.charmm_psf_psb_FF(nitrogen_box_liq ,
                            'nitrogen_box_liq',
                            structure_1 = nitrogen_box_vap  ,
                            filename_1 = 'nitrogen_box_vap',
                            FF_filename ="nitrogen_charmm" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            Bead_to_atom_name_dict = None,
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )
print('Completed: GOMC FF file, and the psf and pdb files for 85 K box')